# Analyses of Methylseq Data in R (on sec server)

Note : For easy installation, create a virtual environnement (eg with conda) with files in a system accessible by sec server (eg /galaxy/home/user/). Install the libraries in that environnement in Desmond server. and run it on sec. (Some libraries are broken right now because of an version conflict, but sec is being updated. Date : May 20th 2020)

## Install packages 

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("illuminaio")
BiocManager::install("IlluminaHumanMethylationEPICanno.ilm10b4.hg19")
BiocManager::install("FlowSorted.CordBlood.EPIC")
BiocManager::install("FlowSorted.Blood.EPIC")  

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("IlluminaHumanMethylationEPICmanifest")
BiocManager::install("IlluminaHumanMethylation450kmanifest")


if (!requireNamespace("glmnet", quietly = TRUE))
    install.packages("glmnet")


In [ ]:
if (!requireNamespace("readr", quietly = TRUE))
    install.packages("readr")
if (!requireNamespace("plyr", quietly = TRUE))
    install.packages("plyr")
if (!requireNamespace("dplyr", quietly = TRUE))
    install.packages("dplyr")
if (!requireNamespace("caret", quietly = TRUE))
    install.packages("caret")
if (!requireNamespace("ggplot2", quietly = TRUE))
    install.packages("ggplot2")
if (!requireNamespace("repr", quietly = TRUE))
    install.packages("repr")

if (!requireNamespace('tidyverse', quietly = TRUE))
    install.packages("tidyverse")

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("GenomeInfoDb")

install.packages("remotes")

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")


if (!requireNamespace("curl", quietly = TRUE))
    install.packages("curl")


BiocManager::install("curl")


BiocManager::install("minfi")

If libcurl is already installed, check that 'pkg-config' is in your
PATH and PKG_CONFIG_PATH contains a libcurl.pc file. If pkg-config
is unavailable you can set INCLUDE_DIR and LIB_DIR manually via:

R CMD INSTALL --configure-vars='INCLUDE_DIR=... LIB_DIR=...

## In R 
### Load Libraries

In [ ]:

library("illuminaio")
library("minfi")
library("minfiData")
library("plyr")
library("readr")
library("dplyr")

library("IlluminaHumanMethylationEPICmanifest")
library("IlluminaHumanMethylationEPICanno.ilm10b4.hg19")
library('IlluminaHumanMethylation450kmanifest')
library("IlluminaHumanMethylation450kanno.ilmn12.hg19")



library("caret")
library("ggplot2")
library("repr")
library("glmnet")
library("GenomeInfoDb")

### Load Data 


Minfi take a folder path in parameter for the import function. The folder must contain .idat files and the sample sheet containing the ID of samples, the corresponding file reference and phenotype. Since the original folder containing the idat files is not writable, I made symbolic links in a folder on my home directory so I can modify the sample sheet to export the data. 

In [ ]:

sheet <- read.metharray.sheet("/nfs/secure/scratch3/child_obesity/analysis/delphinel/data/") #Read Sample Sheet
targets <- read.metharray.exp(targets=sheet) # From the sample shit, import methylation array data

sheetCB <- read.metharray.sheet("/nfs/secure/scratch3/child_obesity/analysis/delphinel/data/")
targetsCB <- read.metharray.exp(targets=sheetCB) # From the sample shit, import methylation array data
targets450kCB =convertArray(targetsCB, outType = "IlluminaHumanMethylation450k",verbose = TRUE)


Most of the functions available in Minfi package have been developped for Illumina450k, the previous generation of IlluminaEPIC. We therefore convert the data to Illumina450k data. I am not sure how this conversion is made, I think it changes the global light intensity of the signal.

In [ ]:
targets450k=convertArray(targets, outType = "IlluminaHumanMethylation450k",verbose = TRUE)


The following command returns the loci that failed detection (signal is an outlier compare to other loci). It is for information, the preprocessing function do it by itself.

In [ ]:
failed_p=detectionP(targets450k, type = "m+u")


Preprocessing of the data : Quantile normalisation, remove sexual chromosomes, remove bad samples,

In [ ]:

preprocessedCB = preprocessQuantile(targets450kCB, fixOutliers = TRUE, removeBadSamples = TRUE, badSampleCutoff = 10, quantileNormalize = TRUE, stratified = TRUE, mergeManifest = FALSE, verbose = TRUE)
phenoData <- pData(preprocessedCB) 

In [ ]:
preprocessed = preprocessQuantile(targets450k, fixOutliers = TRUE, removeBadSamples = TRUE, badSampleCutoff = 10, quantileNormalize = TRUE, stratified = TRUE, mergeManifest = FALSE, verbose = TRUE)
phenoData <- pData(preprocessed) # Extract phenotype data


ERROR: Error in preprocessQuantile(targets450k, fixOutliers = TRUE, removeBadSamples = TRUE, : could not find function "preprocessQuantile"


Addition of SNP infos (from an included database) and remove the position containing one.

In [ ]:
prepro_snp=addSnpInfo(preprocessed)
gset=dropLociWithSnps(prepro_snp, snps=c("SBE","CpG"), maf=0)


prepro_snpCB=addSnpInfo(preprocessedCB)
gsetCB=dropLociWithSnps(prepro_snpCB, snps=c("SBE","CpG"), maf=0)


Mapping the CpG sites to genome

In [ ]:
mapped=minfi::mapToGenome(gset)

Transform data from signal of methylation and unmethylation (M and U) to Beta values : methylation ratio $$(M)\over(M+U+100)$$

In [ ]:
# noOutlierMapped=fixMethOutliers(mapped, K = -3, verbose = FALSE) #compensate outliers

#ratios=minfi::ratioConvert(noOutlierMapped, what ="both", keepCN = TRUE) # convert methylation data into Beta values

Collapsing adjacent CpG sites with the same methylation state into 1 region

In [ ]:
collapsed=cpgCollapse(mapped, what="Beta")

Extracting the position data and merging it to a matrix Containing the Beta values (proportion of methylations). This will be the matrix used for further analyses. We do this for both all positions and merge positions

In [ ]:
workBeta=getBeta(mapped)
colnames(workBeta) #samples
rownames(workBeta) #positions


workcollapsed=getBeta(collapsed$object)
rownames(workcollapsed)=collapsed$blockInfo$indexes
workcollapsed=as.data.frame(workcollapsed)

Addition of the standard deviation(SD) of the methylation ratio of a site across samples in a new column. This will be used when we want to select only most variable positions.

In [ ]:
annots=getAnnotation(mapped, what = "everything", lociNames = NULL, orderByLocation = FALSE, dropNonMapping = FALSE)
names(annots)

In [ ]:
typeof(annots)
typeof(workBeta)

In [ ]:
annots = as.data.frame(annots)
workBeta = as.data.frame(workBeta)

mergedBeta=merge(annots, workBeta, by=0, all=TRUE)

In [ ]:

save(sortedCol, list = "sortedCol", file = "sortedCol.Rdat")


save(c, list = "mergedBeta", file = "Beta_placenta.Rdat")

In [ ]:
write.table(mergedBeta,"Beta_placenta.Rdat.csv", row.names = TRUE , col.names= colnames(mergedBeta), sep="\t")


## Phenotype association Analyses

### Prepare data

Get phenotype data from sample sheet

In [ ]:
testpd=read.csv("Methylseq_sample_sheet_diff_cont_bmi.csv")
cols=colnames(testpd)


cols
[1] "Sample_Name"      "Sample_Well"      "Sample_Plate"     "Sample_Group"
[5] "Pool_ID"          "Sentrix_ID"       "Sentrix_Position"


testpd$Sample_Group
 [1] 4.208000 6.426000 1.282000 3.495000 4.362000 3.274000 2.157000 2.562000
 [9] 0.553000 4.364000 2.625000 1.353000 3.257000 3.293000 4.863000 0.148000
[17] 2.499000 2.063000 3.794000 1.138000 2.222000 4.021000 0.351000 2.375000
[25] 3.888000 2.350000 0.376000 4.003000 7.131000 1.845000 3.222000 5.124000
[33] 3.826000 4.112000 1.967000 4.015000 5.078000 5.410000 5.632000 2.765000
[41] 4.410000 2.683000 3.357000 3.607000 2.611831 5.156000 5.827000 1.489000

continuous_pheno=testpd$Sample_Group

Run DMP finder of minfi to detect differentially methylated positions

In [ ]:
compara=dmpFinder(getBeta(collapsed$object), phenoData$Sample_Group , type ="categorical", qCutoff = 1, shrinkVar = FALSE)
compara_top=dmpFinder(comp_x, phenoData$Sample_Group , type ="categorical", qCutoff = 1, shrinkVar = FALSE)

compara_cont=dmpFinder(getBeta(collapsed$object), continuous_pheno , type ="continuous", qCutoff = 1, shrinkVar = FALSE)


designmatrix=model.matrix(~ continuous_pheno)
dmrs_col <- bumphunter(collapsed$object, design = designmatrix ,cutoff = 0.2, B=0, type="Beta")
dmrs=bumphunter(mapped,design = designmatrix ,cutoff = 0.2, B=0, type="Beta")



In [ ]:
annotation <- getAnnotation(mapped)
names(annotation)

annots=getAnnotation(mapped, what = "everything", lociNames = NULL, orderByLocation = FALSE, dropNonMapping = FALSE)

allpos=array()
for(i in rownames(compara_cont)){
allpos=append(allpos,collapsed$blockInfo$indexes[i])
}



In [ ]:
colcol=append(append('Row',append("Index",mapped$Sample_Name)),"SD")
colnames(sortedCol)=colcol
save(sortedCol, list = "sortedCol", file = "sortedCol.Rdat")

namcol=append("Row.names",mapped$Sample_Name)
namcol=append(namcol,"SD")
colnames(sortedBeta)=namcol
save(c, list = "sortedBeta", file = "sortedBeta.Rdat")

In [ ]:
load("sortedCol.Rdat")
load("sortedBeta.Rdat")


source("sortedCol.Rdat")
source("sortedBeta.Rdat")


Warning message in readChar(con, 5L, useBytes = TRUE):
“cannot open compressed file 'sortedCol.Rdat', probable reason 'No such file or directory'”


ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


In [ ]:
bmi_2yo=read.csv("Methylseq_2yo_bmi.csv", header=TRUE, skip=7)

In [ ]:
phenoData <- pData(mapped)
weightgain=phenoData$Sample_Group

### Run mds

In [ ]:
weightgain=bmi_2yo$Sample_Group
color=rep(NA, length=length(weightgain))
color[which(weightgain=="avg")] = "red"
color[which(weightgain=="high")] = "blue"


ERROR: Error in eval(expr, envir, enclos): object 'bmi_2yo' not found


In [ ]:
for(i in seq(200, 20001, 100)){
topvar=sortedBeta[1:i,]
topcol=sortedCol[1:i,]

dmsmatCol=t(topcol[,-1:-2][,-"SD"])
dcol <- dist(dmsmatCol) # euclidean distances between the rows
fitcol <- cmdscale(dcol,eig=TRUE, k=2) # k is the number of dim
xcol <- fitcol$points[,1]
ycol <- fitcol$points[,2]
jpeg(paste('/nfs/secure/scratch3/child_obesity/analysis/delphinel/mds_2yo_bmi_',as.character(i),'_collapsed.jpg',sep=''))
plot(xcol, ycol, xlab="Coordinate 1", ylab="Coordinate 2",main="MDS", type="n",col=color)
text(xcol, ycol, labels = row.names(dmsmatCol), cex=.7,col=color)
legend(-0.017,-0.008,unique(weightgain),unique(color),pch=1)
dev.off()


dmsmat=t(topvar[,c(-1,-length(colnames(topvar)))])
d <- dist(dmsmat)
fit <- cmdscale(d,eig=TRUE, k=2) 
x <- fit$points[,1]
y <- fit$points[,2]

jpeg(paste('/nfs/secure/scratch3/child_obesity/analysis/delphinel/mds_2yo_bmi_',as.character(i),'.jpg',sep=''))
plot(x, y, xlab="Coordinate 1", ylab="Coordinate 2",main="MDS", type="n",col=color)
text(x, y, labels = row.names(dmsmat), cex=.7,col=color)
legend(-0.02,-0.010,unique(weightgain),unique(color),pch=1)
dev.off()

}

In [ ]:
library("dplyr")
library("tibble")
library("ggplot2")

dmsmatCol=t(sortedCol[,-1:-2][,-"SD"])
dcol <- dist(dmsmatCol) # euclidean distances between the rows

MDSBMI = cmdscale(dcol, eig = TRUE)
plotbar = function(res, m = 9) {
  tibble(eig = res$eig[seq_len(m)], k = seq(along = eig)) %>% ggplot(aes(x = k, y = eig)) + scale_x_discrete("k", limits = seq_len(m)) + theme_minimal() + geom_bar(stat="identity", width=0.5, color="orange", fill="pink")
}

for(i in seq(5, 51, 5))
{
file=paste('/nfs/secure/scratch3/child_obesity/analysis/delphinel/mds_2yo_bmi_m',as.character(i),'_barplot.jpg',sep='')
print(file)
    
jpeg(file)
plotbar(MDSBMI, m = i)
dev.off()
    
}

In [ ]:
MDSeur = tibble( PCo1 = MDSBMI$points[, 1], PCo2 = MDSBMI$points[, 2], labs = rownames(MDSBMI$points))

i=2
file=paste('/nfs/secure/scratch3/child_obesity/analysis/delphinel/own_mds_2yo_bmi_m',as.character(i),'.jpg',sep='')


jpeg(file)
ggplot(MDSeur, aes(x = PCo1, y = PCo2, label = labs)) + geom_point(color = "red") + xlim(-100, 100) + ylim(-1150, 1150) + coord_fixed() + geom_text(size = 4, hjust = 0.3, vjust = -0.5)
dev.off()



In [ ]:
MDSeur = tibble( PCo1 = MDSBMI$points[, 1], PCo2 = MDSBMI$points[, 2], labs = rownames(MDSBMI$points))

i=2
file=paste('/nfs/secure/scratch3/child_obesity/analysis/delphinel/own_mds_2yo_bmi_m',as.character(i),'.jpg',sep='')


jpeg(file)
ggplot(MDSeur, aes(x = PCo1, y = PCo2, label = labs)) + geom_point(color = "red") + xlim(-100, 100) + ylim(-100, 100) + coord_fixed() + geom_text(size = 4, hjust = 0.3, vjust = -0.5)
dev.off()



In [ ]:
bmi_perc=read.csv("Methylseq_perc_bmi.csv", header=TRUE, skip=7)
weightgain=bmi_perc$Sample_Group
color=rep(NA, length=length(weightgain))
color[which(weightgain=="avg")] = "red"
color[which(weightgain=="high")] = "blue"


In [ ]:
for(i in seq(200, 10001, 100)){
topvar=sortedBeta[1:i,]
topcol=sortedCol[1:i,]

dmsmatCol=t(topcol[,-1:-2][,-"SD"])
dcol <- dist(dmsmatCol) # euclidean distances between the rows
fitcol <- cmdscale(dcol,eig=TRUE, k=2) # k is the number of dim
xcol <- fitcol$points[,1]
ycol <- fitcol$points[,2]
jpeg(paste('/nfs/secure/scratch3/child_obesity/analysis/delphinel/mds_perc_bmi_',as.character(i),'_collapsed.jpg',sep=''))
plot(xcol, ycol, xlab="Coordinate 1", ylab="Coordinate 2",main="MDS", type="n",col=color)
text(xcol, ycol, labels = row.names(dmsmatCol), cex=.7,col=color)
legend(-0.017,-0.008,unique(weightgain),unique(color),pch=1)
dev.off()


dmsmat=t(topvar[,c(-1,-length(colnames(topvar)))])
d <- dist(dmsmat)
fit <- cmdscale(d,eig=TRUE, k=2) 
x <- fit$points[,1]
y <- fit$points[,2]
jpeg(paste('/nfs/secure/scratch3/child_obesity/analysis/delphinel/mds_perc_bmi_',as.character(i),'.jpg',sep=''))
plot(x, y, xlab="Coordinate 1", ylab="Coordinate 2",main="MDS", type="n",col=color)
text(x, y, labels = row.names(dmsmat), cex=.7,col=color)
legend(-0.02,-0.010,unique(weightgain),unique(color),pch=1)
dev.off()

}

### GLMnet

In [ ]:
dmsmatCol=t(sortedCol[,-1:-2][,-"SD"])


In [ ]:
glmnet(x, y, family = "binomial", weights, offset = NULL, alpha = 1,
  nlambda = 100, lambda.min.ratio = ifelse(nobs < nvars, 0.01, 1e-04),
  lambda = NULL, standardize = TRUE, intercept = TRUE,
  thresh = 1e-07, dfmax = nvars + 1, pmax = min(dfmax * 2 + 20,
  nvars), exclude, penalty.factor = rep(1, nvars), lower.limits = -Inf,
  upper.limits = Inf, maxit = 1e+05, type.gaussian = ifelse(nvars <
  500, "covariance", "naive"), type.logistic = c("Newton",
  "modified.Newton"), standardize.response = FALSE,
  type.multinomial = c("ungrouped", "grouped"), relax = FALSE,
  trace.it = 0, ...)